In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import lsst.daf.butler as dB
import lsst.cp.verify.notebooks.utils as utils
import lsst.afw.display as afwDisplay

In [ ]:
# This cell contains parameters that can be automatically set via the papermill package.
# Examples:
#  Update parameters in input.ipynb, writing output.ipynb, but do not execute:
#   papermill --prepare-only -p calibType newBias -p cameraName LSSTCam <input> <output>
#  Disable interactive cells in input.ipynb, execute it, and write output.ipynb.
#   papermill -p interactive False <input> <output>
interactive = True

# Which repository to use.
repository = '/repo/embargo/'

# Which calibration type to analyse.
calibType = 'flat'
physical_filter = 'BG40_65mm_1~empty'

# Which camera the calibration is for.
cameraName = 'LATISS'

# Which display to use.
displayBackend = 'astrowidgets'
   
# Which collection the calibration was constructed in.
genCollection = 'LATISS/calib/DM-37587/flatGen-BG40.20230113a'

# Which collection containing the verification outputs.
verifyCollection = 'LATISS/calib/DM-37587/verifyFlat-BG40.20230113a'

In [ ]:
# Get butler and camera
butler = dB.Butler(repository, collections=[verifyCollection, genCollection])
camera = butler.get('camera', instrument=cameraName)

In [ ]:
# Get Run Statistics
runStats = butler.get('verifyFlatStats', instrument=cameraName)
runSuccess = runStats.pop('SUCCESS')

In [ ]:
# Display summary table of tests and failure counts.
utils.failureTable(runStats)

In [ ]:
utils.plotFailures(runStats, camera, scaleFactor=8)

In [ ]:
# Get data for mean(expTime) plot.
ampMeans = {}
expIds = []
for detector in camera:
    ampMeans[detector.getName()] = {}
    for amp in detector.getAmplifiers():
        ampMeans[detector.getName()][amp.getName()] = {'ID': [], 'EXPTIME': [], 
                                                       'MEAN': []}

for exposureId, stats in runStats.items():
    dimensionRecord = butler.registry.queryDimensionRecords('exposure', 
                                                            instrument=cameraName, 
                                                            exposure=exposureId)
    expTime = list(dimensionRecord)[0].exposure_time
    expIds.append(exposureId)
    for detector in camera:
        detId = detector.getId()
        detStats = butler.get('verifyFlatDetStats', instrument=cameraName, 
                              exposure=exposureId, detector=detId)
    
        for amp in detector.getAmplifiers():
            mean = detStats['AMP'][amp.getName()]['MEAN']
            ampMeans[detector.getName()][amp.getName()]['ID'].append(exposureId)
            ampMeans[detector.getName()][amp.getName()]['MEAN'].append(mean)
            ampMeans[detector.getName()][amp.getName()]['EXPTIME'].append(expTime)

In [ ]:
# Plot flux as a function of exposure id, to look for time trends.
for detector in camera:
    detName = detector.getName()
    horizontalSpace = 0.0
    verticalSpace = 150
    plt.figure(figsize=(8, 8))
    fig, axes = plt.subplots(1, 2, figsize=(2 * 8, 8))
    for axis, chunk in zip(axes, [0, 1]):
        for spacer, amp in enumerate(detector.getAmplifiers()):
            axis.scatter(np.array(ampMeans[detName][amp.getName()]['ID']) + 
                         horizontalSpace * spacer,
                         np.array(ampMeans[detName][amp.getName()]['MEAN']) /
                         np.array(ampMeans[detName][amp.getName()]['EXPTIME']) + 
                         verticalSpace * spacer,
                        label=amp.getName())
        axis.set_xlabel("expId")
        axis.set_ylabel("Residual Flux (ADU/s) + Spacer")

        
        if chunk == 0:
            axis.set_xlim(min(expIds), max(expIds)) # This may need editing.
            axis.set_title(f"{calibType} {cameraName} {verifyCollection} ConstructionSet")
        else:
            axis.set_xlim(min(expIds), max(expIds)) # This may need editing.
            axis.set_title(f"{calibType} {cameraName} {verifyCollection} VerificationSet")
        axis.legend()
    plt.show()

In [ ]:
# Plot flux as a function of exposure time, to confirm the residual is flat.
for detector in camera:
    detName = detector.getName()

    horizontalSpace = 0.0
    verticalSpace = 150
    plt.figure(figsize=(8, 8))
    fig, axes = plt.subplots(1, 2, figsize=(2 * 8, 8))
    for axis, chunk in zip(axes, [0, 1]):
        for spacer, amp in enumerate(detector.getAmplifiers()):
            axis.scatter(np.array(ampMeans[detName][amp.getName()]['EXPTIME']) + 
                         horizontalSpace * spacer,
                         np.array(ampMeans[detName][amp.getName()]['MEAN']) /
                         np.array(ampMeans[detName][amp.getName()]['EXPTIME']) + 
                         verticalSpace * spacer,
                        label=amp.getName())
        axis.set_xlabel("exposureTime")
        axis.set_ylabel("Residual Flux (ADU/s) + Spacer")

        if chunk == 0:
            axis.set_title(f"{calibType} {cameraName} {verifyCollection} ConstructionSet")
        else:
            axis.set_title(f"{calibType} {cameraName} {verifyCollection} VerificationSet")
        axis.legend()
    plt.show()

In [ ]:
# This cell may be easier to follow in a new view via the
#     "Create New View for Output" right-click menu.  
afwDisplay.setDefaultBackend(displayBackend)
display = afwDisplay.Display(dims=(1000, 1000))
display.embed()

In [ ]:
# View calibration images:
viewCalibs = interactive
if viewCalibs:
    continueDisplay = True
    for detector in camera:
        detectorId = detector.getId()
        calib = butler.get(calibType, instrument=cameraName, 
                           physical_filter=physical_filter, detector=detectorId)
        calibArray = calib.getImage().getArray()

        # Get simple stats
        q25, q50, q75 = np.percentile(calibArray.flatten(), [25, 50, 75])
        sigma = 0.74 * (q75 - q25)
        print(f"Detector: {detector.getName()} Median: {q50}   Stdev: {sigma}")

        display.mtv(calib)
        display._scale('linear', (q50 - 3.0 * sigma), (q50 + 3.0* sigma), "")

        continueDisplay, skipNumber = utils.interactiveBlock(f"{calibType} {detector.getName()}", 
                                                             {})
        if continueDisplay is False:
            break

In [ ]:
# This block allows the residual images to be scanned for concerns.
blinkResiduals = interactive
if blinkResiduals:
    continueDisplay = True
    skipNumber = 0
    for exposureId, stats in runStats.items():
        for detector in camera:
            if skipNumber > 0:
                skipNumber -= 1
                continue
        
            detId = detector.getId()
            residual = butler.get('verifyFlatProc', instrument=cameraName, 
                                  exposure=exposureId, detector=detId)
            detStats = butler.get('verifyFlatDetStats', instrument=cameraName, 
                                  exposure=exposureId, detector=detId)
            display.mtv(residual)
            display.scale('linear', 'zscale', None)
        
            continueDisplay, skipNumber = utils.interactiveBlock(f"{exposureId} {detector.getName()}", detStats)
            if continueDisplay is False:
                break
        if continueDisplay is False:
            break

In [ ]:
# Additional cells follow here.